# Interesting Points
This notebook reivews the ability to generate interesting points from processed imagery

### Import some libraries, configure Django

In [1]:
import os
import django
import pandas as pd
import geopandas as gpd
import subprocess
from shapely.wkt import loads
from asgiref.sync import sync_to_async
from django.core.management import call_command
from django.contrib.gis.geos import GEOSGeometry

import sys; sys.path.append('../../')
os.environ['DJANGO_SETTINGS_MODULE'] = 'gaia.settings'
django.setup()

from whale.models import ExtractTransformLoad as ETL
from whale.models import PointsOfInterest as POI

### User defined variables

In [2]:
interesting_points = 'C:/gis/apps/libs/generate_interesting_points.py'
input_url = "C:/gis/apps/data/21APR24154045-S1BS-506967344060_01_P006_u08mr32619.tif"

poi_columns = ['id', 'catalog_id', 'vendor_id', 'entity_id', 'sample_idx', 'area', 'deviation', 'epsg_code']

### Generate Interesting Points
This outputs GeoJSON

In [3]:
output_fn = input_url.replace('tif', 'geojson')

subprocess.run([sys.executable, interesting_points, '--input_url', input_url,
                '--output_fn', output_fn, '--method', 'big_window',
                '--difference_threshold', '20', '--overwrite'])

CompletedProcess(args=['C:\\Users\\john.wall\\AppData\\Local\\anaconda3\\envs\\gaia\\python.exe', 'C:/gis/apps/libs/generate_interesting_points.py', '--input_url', 'C:/gis/apps/data/21APR24154045-S1BS-506967344060_01_P006_u08mr32619.tif', '--output_fn', 'C:/gis/apps/data/21APR24154045-S1BS-506967344060_01_P006_u08mr32619.geojson', '--method', 'big_window', '--difference_threshold', '20', '--overwrite'], returncode=1)

### Create Shapefile from GeoJSON for headsup review

In [4]:
gdf = gpd.read_file(output_fn)
output_shp = output_fn.replace('geojson', 'shp')
gdf.to_file(output_shp)

### Review Interesting Points GeoJSON

In [5]:
gdf = gpd.read_file(output_fn)
print(f"The shape of your Geodataframe is: {gdf.shape}\n")
gdf.head()

The shape of your Geodataframe is: (1194, 4)



,id,area,deviation,geometry
0,0,604.714980,25.236259,POINT (409012.120 4629974.586)
1,4,3.092380,70.576653,POINT (410448.701 4629873.244)
2,8,2907.301399,11.100576,POINT (404815.099 4629549.540)
3,9,931.424965,19.001400,POINT (404563.391 4629536.949)
4,10,2.319285,60.941040,POINT (412340.252 4629525.943)


### Add Interesting Points to SpatiaLite Database

In [6]:
def import_poi(geojson_path):
    """ Synchronous Import Points of Interest function.

        Takes the GeoJSON filepath and converts the file path to Vendor ID
            using the panchromatic image as the basis for this (opposed to
            the multispectral). Queries the ExtractTransformLoad (ETL) table
            in SpatiaLite for the relevant Vendor ID object. Reads the GeoJSON
            file to a GeoDataFrame. Updates or creates the Interesting Points
            records from a combination of the ETL and GeoJSON information.

        Print statements support troubleshooting.
    """
    vid = '_'.join(geojson_path.split('/')[-1:][0].split('.')[0].split('_')[:-1]).replace('S1BS', 'P1BS')
    obj = ETL.objects.get(vendor_id=vid)

    epsg_code = geojson_path.split('/')[-1:][0].split('.')[0].split('_')[-1:][0].split('mr')[-1:][0]
    gdf = gpd.read_file(geojson_path)

    for index, row in gdf.iterrows():
        print(f"Processing row: {row['id']}")
        poi, created = POI.objects.update_or_create(
            sample_idx = row['id'],
            defaults={
                'catalog_id': obj.id,
                'vendor_id': obj.vendor_id,
                'entity_id': obj.entity_id,
                'area': row['area'],
                'deviation': row['deviation'],
                'epsg_code': epsg_code,
                'point': row['geometry'].wkt
            }
        )
        print(f"\t{'Created' if created else 'Updated'} POI with id: {poi.sample_idx}\n")

    print('Data imported successfully!')

async def import_poi_async(file_path):
    await sync_to_async(import_poi, thread_sensitive=True)(file_path)

import asyncio

if asyncio.get_event_loop().is_running():
    await import_poi_async(output_fn)
else:
    asyncio.run(import_poi_async(output_fn))

Processing row: 0
	Updated POI with id: 0

Processing row: 4
	Updated POI with id: 4

Processing row: 8
	Updated POI with id: 8

Processing row: 9
	Updated POI with id: 9

Processing row: 10
	Updated POI with id: 10

Processing row: 19
	Updated POI with id: 19

Processing row: 25
	Created POI with id: 25

Processing row: 65
	Updated POI with id: 65

Processing row: 77
	Updated POI with id: 77

Processing row: 106
	Updated POI with id: 106

Processing row: 112
	Updated POI with id: 112

Processing row: 127
	Updated POI with id: 127

Processing row: 131
	Updated POI with id: 131

Processing row: 132
	Updated POI with id: 132

Processing row: 137
	Updated POI with id: 137

Processing row: 141
	Updated POI with id: 141

Processing row: 159
	Created POI with id: 159

Processing row: 162
	Updated POI with id: 162

Processing row: 169
	Updated POI with id: 169

Processing row: 172
	Updated POI with id: 172

Processing row: 176
	Updated POI with id: 176

Processing row: 179
	Updated POI with i

### Confirm that the points were added

In [10]:
objs = await sync_to_async(list)(POI.objects.all())
print(f"Number of POI records in database: {len(objs)}\n")

vid = '_'.join(output_fn.split('/')[-1:][0].split('.')[0].split('_')[:-1]).replace('S1BS', 'P1BS')

geoms = []
attributes = []
for obj in objs:
    if obj.vendor_id == vid:
        attr_dict = {col: getattr(obj, col) for col in poi_columns}
        attributes.append(attr_dict)
    
        geoms.append(GEOSGeometry(obj.point))

gdf = gpd.GeoDataFrame(attributes, geometry = [loads(g.wkt) for g in geoms])
gdf.head()

Number of POI records in database: 109942



,id,catalog_id,vendor_id,entity_id,sample_idx,area,deviation,epsg_code,geometry
0,9,10400100674B2100,21APR24154045-P1BS-506967344060_01_P006,WV320210424154045P01,65,2.473904,99.737572,32619,POINT (412719.624 4628829.873)
1,24,10400100674B2100,21APR24154045-P1BS-506967344060_01_P006,WV320210424154045P01,162,2.319285,51.064480,32619,POINT (407092.315 4627758.099)
2,56,10400100674B2100,21APR24154045-P1BS-506967344060_01_P006,WV320210424154045P01,476,43.293325,15.616094,32619,POINT (411983.823 4625250.730)
3,58,10400100674B2100,21APR24154045-P1BS-506967344060_01_P006,WV320210424154045P01,527,4.638571,9.612699,32619,POINT (412823.282 4624798.926)
4,66,10400100674B2100,21APR24154045-P1BS-506967344060_01_P006,WV320210424154045P01,649,2.319285,14.639173,32619,POINT (411554.468 4624428.868)


In [54]:
vid = '21APR24154044-P1BS-506967344060_01_P005'
cid = '10400100674B2100'
eid = 'WV320210424154044P01'
idd = '96042'

geoms = []
attributes = []
for obj in objs:
    if obj.vendor_id == vid:
        if obj.catalog_id == cid:
            if obj.entity_id == eid:
                #if obj.sample_idx == idd:
                    attr_dict = {col: getattr(obj, col) for col in poi_columns}
                    attributes.append(attr_dict)
                
                    geoms.append(GEOSGeometry(obj.point))

gdf = gpd.GeoDataFrame(attributes, geometry = [loads(g.wkt) for g in geoms])
gdf.head()

,id,catalog_id,vendor_id,entity_id,sample_idx,area,deviation,epsg_code,geometry
0,30,10400100674B2100,21APR24154044-P1BS-506967344060_01_P005,WV320210424154044P01,180,1508.789584,14.986794,32619,POINT (400117.939 4636273.999)
1,38,10400100674B2100,21APR24154044-P1BS-506967344060_01_P005,WV320210424154044P01,268,5.730365,29.603600,32619,POINT (399589.253 4635645.227)
2,41,10400100674B2100,21APR24154044-P1BS-506967344060_01_P005,WV320210424154044P01,339,2.477996,76.877670,32619,POINT (401939.626 4635213.040)
3,48,10400100674B2100,21APR24154044-P1BS-506967344060_01_P005,WV320210424154044P01,393,2.942620,7.251594,32619,POINT (411781.285 4634986.913)
4,82,10400100674B2100,21APR24154044-P1BS-506967344060_01_P005,WV320210424154044P01,1110,2.323121,103.941673,32619,POINT (397956.207 4631420.131)


In [57]:
gdf.query('sample_idx == 96042')

,id,catalog_id,vendor_id,entity_id,sample_idx,area,deviation,epsg_code,geometry


In [58]:
gdf.to_csv('./test.csv')

# End